In [1]:
import rasterio 
from rasterio.merge import merge
import rasterio.plot
from rasterio.windows import Window
from rasterio.plot import show
import glob
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tables
import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier

from tqdm import tqdm
import matplotlib.pyplot as plt

# from osgeo import gdal
project_id = 'moja-328209'
!gcloud config set project {project_id}
!gsutil ls
np.seterr(divide='ignore', invalid='ignore')

Updated property [core/project].
gs://data-myanmar-new/
gs://data-philippines-new/
gs://data-uganda/
gs://philippines-backup-new/


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
src = rasterio.open('/data/uganda/budongo/corrected/2001_resampled.tif')
X_train = src.read()
X_train=X_train.reshape(11,965*965)
src.close()

src = rasterio.open('/data/uganda/budongo/matt/2001_resampled.tif')
y_train = src.read()
y_train=y_train.reshape(1*965*965)
src.close()

for year in range(2002,2016):
  src = rasterio.open('/data/uganda/budongo/corrected/{}_resampled.tif'.format(year))
  X = src.read()
  X=X.reshape(11,965*965)
  X_train=np.hstack([X_train,X])
  src.close()
  del X
  src = rasterio.open('/data/uganda/budongo/matt/{}_resampled.tif'.format(year))
  X = src.read()
  X=X.reshape(1*965*965)
  y_train=np.hstack([y_train,X])
  src.close()
  del X

X_train=X_train.T

In [3]:
print(y_train.mean())
y_train[y_train<=28]=0
y_train[y_train>28]=1
y_train.mean()

29.13350414906563


0.4514532291694632

In [ ]:
rf=RandomForestClassifier(max_depth=50)
rf.fit(X_train,y_train)